## Loading the libraries

In [1]:
## to load the python scripts which consists of pre-defined functions
import sys
sys.path.insert(0, '../scripts')

In [2]:
import pandas as pd
import numpy as np
from helper_functions import *
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
## to show the maximum set rows which makes it easy for visualization
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

## Loading the datasets

In [5]:
%%time
main= pd.read_csv('../Team 4/PLA_DETAIL_IDF_Main_v3.csv',encoding='latin-1')
rating= pd.read_csv('../Team 4/PLA_DETAIL_IDF_Rating.csv')

<timed exec>:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:2: DtypeWarning: Columns (26,30,31,62,81,83,84,105,106,112,115,116,118,119,120,121,122,123,124,125,126,127,135,144,195) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 35.2 s, sys: 7.07 s, total: 42.3 s
Wall time: 44.2 s


### Removing Duplicates - Rating dataframe

In [7]:
## counting the duplicates by using the values_counts()

In [7]:
rating['POLICY_NUMBER'].value_counts().reset_index()

,index,POLICY_NUMBER
0,1750855,11
1,169624,11
2,1308540,9
3,1663832,9
4,1157976,8
...,...,...
1112812,5364912,1
1112813,5364937,1
1112814,5364981,1
1112815,5364990,1


In [8]:
## using cleaning_policy_numbers by using the python script to identify and remove them
rating['POLICY_NUMBER']=cleaning_policy_numbers(rating)

In [9]:
## using the same functioon to remove the duplicates for the main dataframe
main['POLICY_NUMBER']=cleaning_policy_numbers(main)

In [ ]:
# duplicates are identified as properties having multiple inspections

In [10]:
for i in rating.columns:
    k = rating.loc[rating['POLICY_NUMBER']=='01157976',i].value_counts()
    if len(k)>1:
        print(i)

INSPECTION_COMPANY_NAME
INSPECTOR_FIRST_NAME
INSPECTOR_LAST_NAME
INSPECTION_TYPE
INSPECTOR_LICENSE_TYPE
INSPECTOR_LICENSE_NUMBER


In [11]:
inspection_cols = rating.columns[rating.columns.str.startswith('INSP')]
print(inspection_cols)

Index(['INSPECTION_COMPANY_NAME', 'INSPECTOR_FIRST_NAME',
       'INSPECTOR_LAST_NAME', 'INSPECTION_TYPE', 'INSPECTOR_LICENSE_TYPE',
       'INSPECTOR_LICENSE_NUMBER'],
      dtype='object')


In [12]:
# created a new column called NUMBER_OF_INSPECTIONS and got rid of the duplicates
kk = rating['POLICY_NUMBER'].value_counts().reset_index()
kk.rename(columns={'index':'POLICY_NUMBER','POLICY_NUMBER':"NUMBER_OF_INSPECTIONS"},inplace=True)

In [14]:
inspection_cols = np.array(inspection_cols)

In [15]:
duplicate_check = np.append(inspection_cols,'POLICY_NUMBER')
print(duplicate_check)

['INSPECTION_COMPANY_NAME' 'INSPECTOR_FIRST_NAME' 'INSPECTOR_LAST_NAME'
 'INSPECTION_TYPE' 'INSPECTOR_LICENSE_TYPE' 'INSPECTOR_LICENSE_NUMBER'
 'POLICY_NUMBER']


In [16]:
rating.drop_duplicates(subset=duplicate_check).shape

(1830301, 208)

In [17]:
rating.shape

(1830301, 208)

In [18]:
rating.drop_duplicates(subset='POLICY_NUMBER',ignore_index=True,inplace=True,keep='last')

In [22]:
rating = rating.merge(kk,on='POLICY_NUMBER',how='inner')

In [23]:
rating.shape

(1112817, 209)

### Removing duplicates - Main dataframe

In [27]:
main['POLICY_NUMBER'].value_counts()

06311202    8
03882605    8
05591444    7
04182424    7
06724158    7
           ..
07971908    1
07971945    1
07971953    1
07971964    1
07071139    1
Name: POLICY_NUMBER, Length: 1033188, dtype: int64

In [28]:
for i in main.columns:
    kk = main.loc[main['POLICY_NUMBER']=='03882605',i].value_counts()
    if len(kk)>1:
        print(i)

In [29]:
k1 = main.drop_duplicates()

In [30]:
k2= main.drop_duplicates(subset=['POLICY_NUMBER'])

In [32]:
main.shape,k1.shape,k2.shape

((1048575, 41), (1034396, 41), (1033188, 41))

In [ ]:
# removed the email,phone number and any contact related columns that would be helpful for the analysis making it easier

In [36]:
main.drop(columns=['PAYOR','EXPIRATION_DATE','EFFECTIVE_DATE','CURRENT_STATUS','POLICE_TAX_DISTRICT',
                  'FIRE_TAX_DISTRICT','AGENCY_ID','AGENCY_WORK_PHONE','AGENCY_FAX_PHONE','AGENCY_ADDRESS_LINE2','AGENCY_EMAIL_ADDRESS1',
                  'AGENT_EMAIL_ADDRESS','BULK_INTAKE_SUBMISSION',
                  'AGENCY_ADDRESS_LINE1','AGENCY_CITY','AGENCY_STATE','AGENCY_POSTAL_CODE',
                  'AGENT_NAME','AGENT_LICENSE_NUMBER'],inplace=True)

In [38]:
# cleaning the address column and filling the missing values with the empty space
main['PROPERTY_ADDRESS_LINE2'] = main['PROPERTY_ADDRESS_LINE2'].fillna(' ')

In [39]:
main['AGENCY_PHYSICAL_ADDRESS_LINE_2']=main['AGENCY_PHYSICAL_ADDRESS_LINE_2'].fillna(' ')

In [41]:
inds = main.loc[main['AGENCY_PHYSICAL_ADDRESS_LINE_1'].isna()].index
main.drop(index=inds,inplace=True)

In [43]:
# fill missing values with mode
main['POLICY_ENDORSEMENT_NUMBER'] = main['POLICY_ENDORSEMENT_NUMBER'].fillna(main['POLICY_ENDORSEMENT_NUMBER'].mode()[0])

In [44]:
# if there are missing values still, you can remove it
inds = main.loc[main['PROPERTY_ADDRESS_LINE1'].isna()].index
main.drop(index=inds,inplace=True)

In [45]:
# combiningh the address line 1 and address line 2 into one
main['PROPERTY_ADDRESS']=main['PROPERTY_ADDRESS_LINE1']+' '+ main['PROPERTY_ADDRESS_LINE2']
main['AGENCY_ADDRESS']=main['AGENCY_PHYSICAL_ADDRESS_LINE_1']+' '+main['AGENCY_PHYSICAL_ADDRESS_LINE_2']

In [46]:
# removing any useless columns
main.drop(columns=['PROPERTY_ADDRESS_LINE1','PROPERTY_ADDRESS_LINE2','AGENCY_PHYSICAL_ADDRESS_LINE_1','AGENCY_PHYSICAL_ADDRESS_LINE_2'],inplace=True)

In [49]:
# create a new column which uses the PRIOR_POLICY_NUMBER and make it a categorical variable
main['PRIOR_POLICY_NUMBER'] = np.where(main['PRIOR_POLICY_NUMBER'].str.startswith('FRJ'),'Yes','No')

In [53]:
# convert it to a datetime formaty
main['PURCHASED_LEASED_DATE'] = pd.to_datetime(main['PURCHASED_LEASED_DATE'], errors='coerce')

In [54]:
# remove nulls
inds = main.loc[main['PURCHASED_LEASED_DATE'].isna()].index
inds

Int64Index([40559, 136489, 270886, 544020, 892819], dtype='int64')

In [55]:
# drop them
main.drop(index=inds,inplace=True)

In [56]:
main.isna().sum()

POLICY_NUMBER                       0
POLICY_TERM_NUMBER                  0
POLICY_ENDORSEMENT_NUMBER           0
ACCOUNTING_COMPANY                  0
POLICY_TYPE                         0
PRIOR_POLICY_NUMBER                 0
PURCHASED_LEASED_DATE               0
BILLING_METHOD                      0
PAY_PLAN                            0
PROPERTY_CITY                       0
PROPERTY_STATE                      0
PROPERTY_ZIPCODE                    0
PROPERTY_COUNTY                     0
AGENCY_NAME                         0
AGENCY_FEIN                         0
AGENCY_PHYSICAL_ADDRESS_CITY        0
AGENCY_PHYSICAL_ADDRESS_STATE       0
AGENCY_PHYSICAL_ADDRESS_ZIP_CODE    0
PROPERTY_ADDRESS                    0
AGENCY_ADDRESS                      0
dtype: int64

In [57]:
# apply string functions to the address based columns and captialize them
main[['PROPERTY_ADDRESS','PROPERTY_CITY','PROPERTY_COUNTY','PROPERTY_STATE']]  = main[['PROPERTY_ADDRESS','PROPERTY_CITY','PROPERTY_COUNTY','PROPERTY_STATE']].applymap(lambda x: x.strip().capitalize())

In [58]:
# apply same as above for the agency columns
main[['AGENCY_ADDRESS','AGENCY_PHYSICAL_ADDRESS_CITY','AGENCY_PHYSICAL_ADDRESS_STATE']]  = main[['AGENCY_ADDRESS','AGENCY_PHYSICAL_ADDRESS_CITY','AGENCY_PHYSICAL_ADDRESS_STATE']].applymap(lambda x: x.strip().capitalize())

In [61]:
## clean the zipcode by removing the FIPS code and converting it to String datatype
main['PROPERTY_ZIPCODE'] = main['PROPERTY_ZIPCODE'].apply(lambda x: x.split('-')[0])
main['AGENCY_PHYSICAL_ADDRESS_ZIP_CODE'] = main['AGENCY_PHYSICAL_ADDRESS_ZIP_CODE'].astype('str')
main['AGENCY_PHYSICAL_ADDRESS_ZIP_CODE'] = main['AGENCY_PHYSICAL_ADDRESS_ZIP_CODE'].apply(lambda x: x.split('.')[0])

In [62]:
# make sure they zipcode is of length 5 without loss of information
main[['PROPERTY_ZIPCODE','AGENCY_PHYSICAL_ADDRESS_ZIP_CODE']]= main[['PROPERTY_ZIPCODE','AGENCY_PHYSICAL_ADDRESS_ZIP_CODE']].applymap(lambda x : x.zfill(5))

In [63]:
main['AGENCY_PHYSICAL_ADDRESS_ZIP_CODE'].nunique(),main['PROPERTY_ZIPCODE'].nunique()

(894, 1004)

In [64]:
main.loc[~main['PROPERTY_ZIPCODE'].isin(main['AGENCY_PHYSICAL_ADDRESS_ZIP_CODE']),'PROPERTY_ZIPCODE'].nunique()

303

In [65]:
# convetrt agencies name to lower case
main['AGENCY_NAME'] = main['AGENCY_NAME'].apply(lambda x: x.strip().lower())

In [66]:
main['AGENCY_NAME'].nunique(),main['AGENCY_FEIN'].nunique()

(5188, 5173)

In [67]:
main['AGENCY_FEIN'] = main['AGENCY_FEIN'].astype('str')
main['AGENCY_FEIN'] = main['AGENCY_FEIN'].apply(lambda x: x.split('.')[0])

In [68]:
main[['AGENCY_FEIN','AGENCY_NAME']] = main[['AGENCY_FEIN','AGENCY_NAME']].applymap(lambda x: x.strip())

In [70]:
# save as version_1
main.to_csv('meth_v1.csv',index_label=False)

### Load again from methodology_v2.csv and merge it with the Rating dataframe

In [71]:
rating.shape

(1112817, 209)

In [72]:
main=pd.read_csv('./meth_v2.csv')

In [73]:
main.shape

(1048551, 20)

In [74]:
main['POLICY_NUMBER'].dtypes,rating['POLICY_NUMBER'].dtypes

(dtype('int64'), dtype('O'))

In [75]:
# convert to appropriate datatypes
main['POLICY_NUMBER'] = main['POLICY_NUMBER'].astype('str')
main['POLICY_NUMBER'] = main['POLICY_NUMBER'].apply(lambda x: x.zfill(8))

In [77]:
main.drop(columns=['POLICY_TERM_NUMBER','POLICY_ENDORSEMENT_NUMBER','ACCOUNTING_COMPANY','POLICY_TYPE'],inplace=True)

In [78]:
# drop duplicates from the main dataframe
main.drop_duplicates(subset='POLICY_NUMBER',ignore_index=True,inplace=True)

In [79]:
main.shape

(1033164, 16)

In [81]:
# maerge on rating
df = rating.merge(main,on='POLICY_NUMBER',how='inner')

In [82]:
df.shape

(1033164, 224)

In [84]:
for i in df.columns:
    k = df.loc[df['POLICY_NUMBER']=='03882605',i].unique()
    if len(k)>1:
        print(i)

In [86]:
df.drop(columns=['PRIOR_POLICY_NUMBER_x'],inplace=True)

In [87]:
inspection_cols = df.columns[df.columns.str.startswith('INSP')]
inspection_cols

Index(['INSPECTION_COMPANY_NAME', 'INSPECTOR_FIRST_NAME',
       'INSPECTOR_LAST_NAME', 'INSPECTION_TYPE', 'INSPECTOR_LICENSE_TYPE',
       'INSPECTOR_LICENSE_NUMBER'],
      dtype='object')

In [88]:
inspection_cols =np.array(inspection_cols)
inspection_cols

array(['INSPECTION_COMPANY_NAME', 'INSPECTOR_FIRST_NAME',
       'INSPECTOR_LAST_NAME', 'INSPECTION_TYPE', 'INSPECTOR_LICENSE_TYPE',
       'INSPECTOR_LICENSE_NUMBER'], dtype=object)

In [89]:
inspection_cols = np.append(inspection_cols,'FOUR_POINT_INSPECTION_DATE')

In [90]:
df.drop(columns=inspection_cols,inplace=True)

In [91]:
# drop the columns with missing values greater than 40%
cols= get_missing_values(df,40)
cols

array(['RENTAL_FREQUENCY', 'OWNER_OCCUPIED_PCT', 'POOL_FENCE_HEIGHT',
       'MH_HAS_SKIRTING', 'FLOOD_INSURER_NAME', 'FLOOD_POLICY_NUMBER',
       'FLOOD_POLICY_EFFECTIVE_DATE', 'FLOOD_BUILDING_LIMIT',
       'FLOOD_CONTENTS_LIMIT', 'FHCF_STATUS', 'IS_PRIMARY_HEAT_PORTABLE',
       'IS_PRIMARY_HEAT_OPEN_FLAME',
       'IS_PRIMARY_HEAT_FIREPLACE_OR_WOOD_STOVE', 'REMAINING_ROOF_LIFE',
       'PLUMBING_UPDATE_YEAR', 'HEATING_UPDATE_YEAR',
       'WIRING_UPDATE_YEAR', 'CLASS_C_TYPE', 'FBC_WIND_SPEED',
       'FBC_WIND_DESIGN', 'WINDSTORM_PROTECTIVE_DEVICES',
       'VALUATION_SOURCE', 'ALTERNATE_VALUATION_REPLACEMENT_COST',
       'ACTUAL_CASH_VALUE', 'STATED_VALUE_AMOUNT',
       'STATED_VALUATION_SOURCE', 'ALTERNATE_VALUE_AMOUNT',
       'RENOVATION_OCCUPIED', 'RENOVATION_COMPLETION_DATE',
       'BURGLAR_ALARM_TYPE', 'MOBILE_HOME_MANUFACTURER',
       'MOBILE_HOME_SECTION_LENGTH_FT', 'MOBILE_HOME_SECTION_WIDTH_FT',
       'MOBILE_HOME_SERIAL_NUMBER', 'MOBILE_HOME_INSTALATION_DATE',
   

In [92]:
len(cols)

48

In [93]:
df.drop(columns=cols,inplace=True)

In [94]:
df.shape

(1033164, 168)

In [ ]:
df.to_csv('meth_v3.csv',index_label=False)

In [1]:
## Load the Data_cleaning_v2.ipynb with meth_v3.csv file